# Expected Goals - Hvordan bliver en xG model lavet?

`Expected Goals (xG)` har i de seneste år været en meget omdiskuteret del af fodbold analyse, og er efterhånden blevet noget alle fodboldfans kender. Men xG er, som mange andre elementer i maskin læring, en "black boks". 
Hvordan kan to forskellige udbydere af xG f.eks. have henholdsvis 0.8 xG og 1.27 xG? (https://twitter.com/tapinfodbold/status/1280186019309072384)

Jeg vil der i denne artikel vise hvordan en basal xG model bliver lavet for, at give en bedre forståelse for de mange fodboldfans. Jeg gør brug af programmeringssproget `Python`. Mange bruger også `R`, men hvilket sprog der er bedst er det meget debat om. Men i det store billede gør det ikke den store forskel.

### Forbedredelse af data og enviroment

Først importerer vi de bibloteker (libraries), som vi skal gøre brug til at bygge vores model.

In [3]:
#importere bibloteker
import requests #få data gennem URL, fra internettet
import pandas as pd #bruges til at nemmere at manipulere, tilgå og opbevare vores data
import numpy as np #bruges til matematiske beregninger/manipulationer på vores data.

Jeg vil gøre brug at åben `event data` fra StatsBomb, og bruge skud fra VM 2018 til at træne vores model.

In [4]:
#URl's vi vil bruge til at få adgang til vores data
base_url = "https://raw.githubusercontent.com/statsbomb/open-data/master/data/"
comp_url = base_url + "matches/{}/{}.json"
match_url = base_url + "events/{}.json"

`base_url` er den første hjemmeside hvor dataen ligger, `comp_url` er her hvor information om hver kamp ligger, såsom hvilke hold der spiller, hvilken dommer, kamp ID'et osv. `match_url` indeholder de begivenheder (events) som der skete i kampen herunder skud, afleveringer, hvis en spiller går i pres, osv.
Herefter bliver {} udfyldt med funktionen `.format()` med det givne stævne ID og Sæson ID. VM 2018 har f.eks. stævne ID 43 og Sæson ID 3.



In [9]:
def parse_data(competition_id, season_id):
    matches = requests.get(url=comp_url.format(competition_id, season_id)).json() #få alt kamp data fra den specifikke turnering
    match_ids = [match["match_id"] for match in matches] #tag alle kamp ID's fra hver kamp i turneringen
    
    #vi looper gennem listen med kamp ID's og få begivenhederne fra hver kamp
    all_events = []
    for match_id in match_ids:
        
        events = requests.get(url=match_url.format(match_id)).json() #udtrækker event data filen fra hver kamp ID
        
        shots = [event for event in events if event["type"]["name"] == "Shot"] #udtræk skud fra event dataen
        #vi kan nu udtrække de egenskaber (features) fra skudene
        for shot in shots:
            features = {
                "x": shot["location"][0],
                "y": shot["location"][1],
                "head": 1 if shot["shot"]["body_part"]["name"] == "Head" else 0,
                "phase": shot["shot"]["type"]["name"], #from which phase the shot came from
                "outcome": 1 if shot["shot"]["outcome"]["name"] == "Goal" else 0,
                "statsbomb_xg": shot["shot"]["statsbomb_xg"]
                
            }
        all_events.append(features)
    
    
    return pd.DataFrame(all_events)

Vi trækker først alt kamp dataen ud fra hver kamp i den specifikke turnering og sæson, hvilket i vores tilfælde er VM 2018 (43 og 3).

Herefter gør vi brug af en _list comprehension_ som looper gennem kamp dataten og laver en liste med hver kamp ID fra alle kampene. Dette kamp ID skal bruges til at udtrække event dataen, som sker i næste skridt.

Her klargør vi en tom liste som kan indeholde den event data i vores interesse, og så looper vi igennem kamp ID'erne og for hver kamp ID udtrækker vi event dataen som ligger i `match_url`.

Det gør i stand til at trække alle skudene ud af event dataen. Vi looper derefter i gennem skudene og udvælger de bestemte egenskaber ved skudene, som vi mener har indflydelse på skudet resulterer i et mål. Det er i vores tilfælde:
- `x, y koordinater`, som gør i stand til at udregne distance til mål og hvilken vinkel skudet har til målet.
- `head`, hvilket er om skudet var et hovedstød (1) eller ikke (0).
- `phase`, hvilet betegner om skudet kom efter et hjørnespark, frispark, åbent spil, osv.
- `outcome` er skudets resultat. Endte skudet med mål (1) eller ikke med mål (0).
- `statsbomb_xg`, hvilket er den xG som StatsBomb model har tilgivet skudet. Dette vil blive brugt til at sammenligne og evaluere vores model med en meget kompleks xG model.

Disse egenskaber er nu lagt ind i vores `all_events` liste og derefter retuneret i en `PandasDatFrame` som gør at vi kan manipulere og arbejde bedre med vores data.

-----------------------

Vi kan nu udtrække dataen fram VM 2018, som turnering ID 43 og sæson 3. Herefter bruger vi vores `parse_data` funktion med disse to argumenter og dataen fra VM 2018 bliver gemt i en `Pandas Dataframe` i variablen `df`.

In [15]:
competition_id = 43
season_id = 3
df = parse_data(competition_id, season_id)
df.head()

,x,y,head,phase,outcome,statsbomb_xg
0,108.0,40.0,0,Penalty,1,0.760000
1,103.0,39.0,0,Open Play,0,0.065057
2,92.0,50.0,0,Open Play,0,0.017895
3,111.0,52.0,0,Open Play,0,0.175656
4,115.0,42.0,0,Open Play,1,0.842828


Dette er sådan vores de fem første rækker af vores data ser ud. Hvor vi har de forskellige features 

Vi kan nu udregne skudets distance til mål (`distance_to_goal`) og dets vinkel til mål (`goal_angle`), hvilket er to meget gode prædiktorer for om skudet ender med et mål. 

In [10]:
#funktion som udregner skudets distance til mål
def distance_to_goal(origin):
    dest = np.array([120., 40.])
    return np.sqrt(np.sum((origin - dest) ** 2))

In [11]:
#funktion som udregner skudets vinkel til mål
def goal_angle(origin):
    p0 = np.array((120., 36.))  # Left Post
    p1 = np.array(origin, dtype=np.float)
    p2 = np.array((120., 44.))  # Right Post

    v0 = p0 - p1
    v1 = p2 - p1

    angle = np.abs(np.math.atan2(np.linalg.det([v0, v1]), np.dot(v0, v1)))
    
    return angle

Disse beregninger er taget fra Devin Pleuler Soccer Analytics Handbook (https://github.com/devinpleuler/analytics-handbook).


Vi kan nu lægge disse to beregninger til vores egenskaber ved skudet og lægge som en kolonne i vores dataframe.

Vi bruger en teknik der hedder `broadcasting`, som er meget effektiv på store datasets. (https://stackoverflow.com/questions/29954263/what-does-the-term-broadcasting-mean-in-pandas-documentation)
Samtidig gør vi brug af `lambda` funktion, hvilket gør at vi ikke skal definere en helt ny funktion.

In [12]:
df['distance_to_goal'] = df.apply(lambda row: distance_to_goal(row[['x', 'y']]), axis=1)
df['goal_angle'] = df.apply(lambda r: goal_angle(r[['x', 'y']]), axis=1)

In [13]:
df.head()

,play_pattern,head,x,y,phase,outcome,statsbomb_xg,distance_to_goal,goal_angle
0,Regular Play,0,104.0,37.0,Open Play,0,0.068793,16.278821,0.474829
1,From Corner,1,114.0,44.0,Open Play,0,0.032231,7.211103,0.927295
2,From Goal Kick,1,111.0,44.0,Open Play,0,0.095063,9.848858,0.726642
3,From Throw In,0,113.0,46.0,Open Play,1,0.635004,9.219544,0.681771
4,From Throw In,0,97.0,30.0,Open Play,0,0.016997,25.079872,0.291606


# Træning, test og evalution af vores model
Vi har ny klargjort vores data, så vores model er klar til at blive trænet.

In [12]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier